# _Проект по анализу данных_   ( ◕▿◕ )

состав команды:
- Григорян Тигран
- Томинец Александ
- Шошкин Артемий

В рамках проекта наша команда решила рассмотреть изменения за промежуток чуть больший, чем две декады в NBA. Проследить изменения и разрушить некоторые заблуждения, ветающие в баскетбольном сообществе в последнее время.

### Пункт 1: парсинг и сбор данных (ᗒᗣᗕ)՞

Данные мы собирали с непокладистого basketball-reference. Изначально было множество попыток собрать данные через суповые методы (BeautifulSoup) через подставных агентов (fake_useragent), а также библиотеку HTMLSession. Все наши попытки парсинга с помощью этих методов представлены в других файлах, чтобы не захламлять чистовик (o･ω･o). В конечном итоге мы нашли данные на кегле, которые пока не использовали для проверки первых гипотез, но впоследствии они нам понадобятся (данные и ссылки на них будут в папке "данные для проекта"), а также прошлись циклом по вышеупомянутому basketball-reference и с помощью pd.read_html собирали таблички (код представлен ниже). Для проверки второй гипотезы, которая будет представлена чуть ниже, мы воспользовались данными с сайта spotrac о ста самых высокооплачиваемых баскетболистах на данный момент (первичные результаты проверки гипотезы будут в файле "2 гипотеза").

### Пункт 2: составление гипотез (*≧ω≦*)

Проверять мы будем следующие утверждения:
- в современном баскетболе лучше бросают трехочковые и их влияние на игру стало гораздо больше?
    - визуализируем процент попадания трехочковых по годам и процент очков, заработанных после попадания трехочковыми, от общего числа очков за игру 
- темнокожих игроков в NBA ущемляют (лол)
    - своего рода маленькой социологическое расовое исследование, ограничивающееся сравнением заработных плат игроков разных рас
- нумерелогия - это лженаука  
    - насколько зависят показатели игрока от знака зодиака
    - выявим в каких показателях каждый знак зодиака силен
- попытка предсказать победителя сезона 23/24 
    - попытка выявить победителя в нынешнем сезоне на основе показателей команды за последние годы
- самое важное в баскетболе - это забивать много очков 
    - проверим какой показатель больше всего влияет на победу в матче
    

### Пункт 3: реализация всего вышеперечисленного   (＞﹏＜)

Пока так или иначе получилось реализовать первые два пункта. По мере работы с проектом могут быть изменения и в ныне завершенных пунктах.

_спасибо за внимание!_

### _Пункт 1: парсинг и сбор данных_ (ᗒᗣᗕ)

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

In [3]:
df_00_23 = dict()

# cобираем данные для 1-ой гипотезы
for year in tqdm(range(2000, 2024)):
    if year < 2016:
        link = f'https://www.basketball-reference.com/leagues/NBA_{year}.html'
        df_00_23.update({year : pd.read_html(link)[2]})
        time.sleep(5) 
    else:
        link = f'https://www.basketball-reference.com/leagues/NBA_{year}.html'
        df_00_23.update({year : pd.read_html(link)[4]})
        time.sleep(5) 

100%|██████████| 24/24 [02:40<00:00,  6.70s/it]


In [4]:
# меняем все неактуальные названия команд на нынешние

for year in range(2000, 2024):

    if len(df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Seattle SuperSonics']) > 0:
        df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Seattle SuperSonics'] = 'Oklahoma City Thunder'

    if len(df_00_23[year]['Team'][df_00_23[year]['Team'] == 'New Jersey Nets']) > 0:
        df_00_23[year]['Team'][df_00_23[year]['Team'] == 'New Jersey Nets'] = 'Brooklyn Nets'

    if len(df_00_23[year]['Team'][df_00_23[year]['Team'] == '']) > 0:
        df_00_23[year]['Team'][df_00_23[year]['Team'] == ''] = ''

    if len(df_00_23[year]['Team'][df_00_23[year]['Team'] == 'New Orleans Hornets']) > 0 or len(df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Oklahoma City Hornets']):
        df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Oklahoma City Hornets'] = 'New Orleans Pelicans'
        df_00_23[year]['Team'][df_00_23[year]['Team'] == 'New Orleans Hornets'] = 'New Orleans Pelicans'
        

    if len(df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Vancouver Grizzlies']) > 0:
        df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Vancouver Grizzlies'] = 'Memphis Grizzlies'
        

# убераем лишние символы в названии команд

for year in df_00_23:
    df_00_23[year].Team = [i.replace('*', '') for i in df_00_23[year].Team.values]

# добавляем колонку с годами
for j in df_00_23:
    df_00_23[j]['year'] = j

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12540\1538921252.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_00_23[year]['Team'][df_00_23[year]['Team'] == 'New Jersey Nets'] = 'Brooklyn Nets'
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12540\1538921252.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_00_23[year]['Team'][df_00_23[year]['Team'] == 'Vancouver Grizzlies'] = 'Memphis Grizzlies'
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12540\1538921252.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [5]:
team_res = dict()

# немного преобразуем наши данные и сохраним их в словаре выше, где ключом будет название команды, а значением ее результаты
for team in df_00_23[year]['Team']: 
    _ = []
    for year in range(2000, 2024):
        _.append(df_00_23[year].loc[df_00_23[year]['Team'] == team])
        team_res.update({team : _})

In [6]:
df = pd.DataFrame()

# собираем значения из словаря в один датасет
for i in team_res:
    for j in range(len(team_res[i])):
        df = pd.concat([df, team_res[i][j]])

In [8]:
# сохраним датасет в папке с данными

df.to_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\df.csv")

In [17]:
# собираем с сайта табличку, обрабатываем ее и сохраняем в csv для 2-ой гипотезы

salary_rank = pd.read_html('https://www.spotrac.com/nba/rankings/')[0]
salary_rank['Team'] = [i[-4:] for i in salary_rank.Player.values]
salary_rank['Player'] = [i[:-5] for i in salary_rank.Player.values]
salary_rank['cap hit'] = [
    int(s.replace("$", "").replace(",", "")) for s in salary_rank['cap hit'].values
    ]
column_name = ['Player', 'Team', 'Position', 'Age', 'cap hit']
salary_rank = salary_rank[column_name]
salary_rank.to_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\top_100_salary.csv")

In [9]:
# посмотри также на скачанные с кегля данные

games = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\games.csv.zip")
games_details = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\games_details.csv.zip")
players = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\players.csv")
ranking = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\ranking.csv.zip")
teams = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\teams.csv")

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12540\3134026677.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details = pd.read_csv(r"C:\Users\HUAWEI\hw\проект по анализу данных\данные\games_details.csv.zip")


In [10]:
games

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
26647,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
26648,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
26649,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [11]:
games_details

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668623,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,19,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
668624,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,23,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
668625,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,15,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
668626,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,19,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


In [12]:
players

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019
...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009
7224,Warren Carter,1610612752,201999,2009
7225,Bennet Davis,1610612751,201834,2009
7226,Brian Hamilton,1610612751,201646,2009


In [13]:
ranking

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210337,1610612765,0,22013,2014-09-01,East,Detroit,82,29,53,0.354,17-24,12-29,NaN
210338,1610612738,0,22013,2014-09-01,East,Boston,82,25,57,0.305,16-25,9-32,NaN
210339,1610612753,0,22013,2014-09-01,East,Orlando,82,23,59,0.280,19-22,4-37,NaN
210340,1610612755,0,22013,2014-09-01,East,Philadelphia,82,19,63,0.232,10-31,9-32,NaN


In [14]:
teams

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
5,0,1610612743,1976,2019,DEN,Nuggets,1976,Denver,Pepsi Center,19099.0,Stan Kroenke,Tim Connelly,Michael Malone,No Affiliate
6,0,1610612745,1967,2019,HOU,Rockets,1967,Houston,Toyota Center,18104.0,Tilman Fertitta,Daryl Morey,Mike D'Antoni,Rio Grande Valley Vipers
7,0,1610612746,1970,2019,LAC,Clippers,1970,Los Angeles,Staples Center,19060.0,Steve Ballmer,Michael Winger,Doc Rivers,Agua Caliente Clippers of Ontario
8,0,1610612747,1948,2019,LAL,Lakers,1948,Los Angeles,Staples Center,19060.0,Jerry Buss Family Trust,Rob Pelinka,Frank Vogel,South Bay Lakers
9,0,1610612748,1988,2019,MIA,Heat,1988,Miami,AmericanAirlines Arena,19600.0,Micky Arison,Pat Riley,Erik Spoelstra,Sioux Falls Skyforce
